In [ ]:
import pandas as pd 
import os
import sys

In [ ]:
melb_df = pd.read_csv(sys.path[0] + '/data/melb_data_fe.csv', sep = ',')
melb_cp = melb_df.copy() 

In [ ]:
display(melb_cp.info())

In [ ]:
melb_cp['Date'] = pd.to_datetime(melb_cp['Date'], dayfirst=True)

1.1

In [ ]:
def get_quarter(month):
    if month in range(1,4):
        return 1
    elif  month in range(4,7):
        return 2
    elif month in range(7,10):
        return 3
    elif month in range(10,13):
        return 4
    

In [ ]:
melb_cp['Quarter'] = melb_cp['Date'].dt.month.apply(get_quarter)

Количество проданных квартир по четвертям года

In [ ]:
display(melb_cp['Quarter'].value_counts())

1.2

In [ ]:
melb_cp2 = melb_df.copy()

In [ ]:
columns_to_change = []
ban_list = ['Date', 'Bedroom', 'Rooms', 'Car', 'Bathroom']

for columnName in melb_cp2.columns.tolist():
    if len(melb_cp2[columnName].value_counts().nlargest(150).index.tolist()) < 150 and columnName not in ban_list:
        columns_to_change.insert(0, columnName)

#display(columns_to_change)

for columnName in melb_cp2.columns.tolist():
    if columnName in columns_to_change:
        melb_cp2[columnName] = melb_cp2[columnName].astype('category')
        
display(melb_cp2.info())

Сортировка столбца. Сортировка по убыванию(ascending=False)
Обновление индексов(ignore_index)

In [ ]:
melb_cp.sort_values(by='AreaRatio', ascending=False, ignore_index=True)
display(melb_cp.iloc[1558])
display(melb_cp.info())

In [ ]:
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2

result = melb_df[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'], 
    ascending=[True, False], 
    ignore_index=True)
print(result.iloc[18])

In [ ]:
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
print(melb_df[mask1&mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price'])

Группировка данных. Вывод по убыванию

In [ ]:
display(melb_df.groupby('Rooms')['Price'].agg('mean').sort_values(ascending=False))

In [ ]:
display(melb_df.groupby('Regionname')['Lattitude'].agg('std').sort_values(ascending=True))

Какая риелторская контора имеет наименьшую общую выручку за период

In [ ]:
melb_cp3 = melb_df.copy()
melb_cp3['Date'] = pd.to_datetime(melb_cp3.Date, dayfirst=True)
melb_date = melb_cp3[(melb_cp3['Date'] >= pd.to_datetime('2017-05-01')) & (melb_cp3['Date'] <= pd.to_datetime('2017-09-01'))]
#mask1 = 
display(melb_date.groupby('SellerG')['Price'].agg('sum').sort_values(ascending=True))

Простейшая сводная таблица

In [ ]:
print(melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median())

In [ ]:
print(melb_df.groupby(['Rooms', 'Type'])['Price'].mean())

In [ ]:
print(melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack())

Тоже самое, но методом pivot_table

In [ ]:
print(melb_df.pivot_table(
    values='Price', # Данные в таблице
    index='Rooms', # Индексы сводной таблицы
    columns='Type', # имена столбцов: категории в 'Type'
    fill_value=0, # пропуски запонить нулями
).round())

In [ ]:
print(melb_df.pivot_table(
    values='Price', # Данные в таблице
    index='Regionname', # Индексы сводной таблицы
    columns='Weekend', # имена столбцов: категории в 'Type' 
    fill_value=0, # пропуски запонить нулями
    aggfunc='count'
).round())

In [ ]:
print(melb_df.pivot_table(
    values='Price', # имя столбца, к которому будут применены агрегирующие функции
    index=['Regionname','Type'], # Индексы сводной таблицы
    columns='Weekend', # имена столбцов: категории в 'Type' 
    fill_value=0, # пропуски запонить нулями
    aggfunc='count' # список агрегирующих функций
).round())

In [ ]:
table = melb_df.pivot_table(
    values='BuildingArea', # имя столбца, к которому будут применены агрегирующие функции
    index='Type', # Индексы сводной таблицы
    columns='Rooms', # имена столбцов: категории в 'Type' 
    fill_value=0, # пропуски запонить нулями
    aggfunc='median' # список агрегирующих функций
)
print(table)

In [ ]:
table = melb_df.pivot_table(
    values='Price', # имя столбца, к которому будут применены агрегирующие функции
    index='SellerG', # Индексы сводной таблицы
    columns='Type', # имена столбцов: категории в 'Type' 
    fill_value=0, # пропуски запонить нулями
    aggfunc='median' # список агрегирующих функций
)
max_price = table['unit'].max()

#print(table)
display(table[table['unit'] == max_price])
display(table[table['unit'] == max_price].index)